In [ ]:
%pip install pandas torch tqdm transformers

In [ ]:
# -- iPython Config --
from IPython import get_ipython
if "IPython.extensions.autoreload" not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic("load_ext", "autoreload")
else:
    get_ipython().run_line_magic("reload_ext", "autoreload")
%autoreload 2

# -- System and Path --
import os
import sys
REPO_PATH = os.path.abspath(os.path.join(".."))
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
print(f"REPO_PATH: {REPO_PATH}")
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# -- Imports --
import os
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
import numpy as np

In [ ]:
# -- Configuration --
class Config:
    """Holds all configuration parameters for the script."""
    def __init__(self):
        self.repo_path = os.path.abspath(os.path.join(".."))
        self.data_dir = self.repo_path
        self.tokenized_data_dir = os.path.join(self.repo_path, "tokenized")
        self.model_name = "GSAI-ML/LLaDA-8B-Instruct"
        self.batch_size = 2
        self.lr = 1e-5
        self.num_epochs = 1
        self.seed = 42
        self.mask_token_id = 126336
        self.device = self._select_device()

    def _select_device(self):
        """Selects the best available device (CUDA, MPS, or CPU)."""
        if torch.cuda.is_available():
            device = "cuda"
        elif torch.backends.mps.is_available():
            device = "mps"
        else:
            device = "cpu"
        print(f"Using device: {device}")
        return device
config = Config()

# Inference

In [ ]:
# -- Generation Helper Functions --
def add_gumbel_noise(logits, temperature):
    """Adds Gumbel noise to logits for sampling."""
    if temperature == 0:
        return logits
    logits = logits.to(torch.float64)
    noise = torch.rand_like(logits, dtype=torch.float64)
    gumbel_noise = (-torch.log(noise)) ** temperature
    return logits.exp() / gumbel_noise

def get_num_transfer_tokens(mask_index, steps):
    """Calculates the number of tokens to transfer at each generation step."""
    mask_num = mask_index.sum(dim=1, keepdim=True)
    base = mask_num // steps
    remainder = mask_num % steps
    num_transfer_tokens = torch.zeros(mask_num.size(0), steps, device=mask_index.device, dtype=torch.int64) + base
    for i in range(mask_num.size(0)):
        num_transfer_tokens[i, :remainder[i]] += 1
    return num_transfer_tokens

@torch.no_grad()
def generate(model, prompt, steps=128, gen_length=128, block_length=128, temperature=0.,
             cfg_scale=0., remasking='low_confidence', mask_id=126336):
    """Generates text using the trained model."""
    x = torch.full((1, prompt.shape[1] + gen_length), mask_id, dtype=torch.long).to(model.device)
    x[:, :prompt.shape[1]] = prompt.clone()
    prompt_index = (x != mask_id)

    assert gen_length % block_length == 0
    num_blocks = gen_length // block_length
    assert steps % num_blocks == 0
    steps_per_block = steps // num_blocks

    for num_block in range(num_blocks):
        start_idx = prompt.shape[1] + num_block * block_length
        end_idx = start_idx + block_length
        block_mask_index = (x[:, start_idx:end_idx] == mask_id)
        num_transfer_tokens = get_num_transfer_tokens(block_mask_index, steps_per_block)

        for i in range(steps_per_block):
            mask_index = (x == mask_id)
            if cfg_scale > 0:
                un_x = x.clone()
                un_x[prompt_index] = mask_id
                x_ = torch.cat([x, un_x], dim=0)
                logits = model(x_).logits
                logits, un_logits = torch.chunk(logits, 2, dim=0)
                logits = un_logits + (cfg_scale + 1) * (logits - un_logits)
            else:
                logits = model(x).logits

            logits_with_noise = add_gumbel_noise(logits, temperature)
            x0 = torch.argmax(logits_with_noise, dim=-1)

            if remasking == 'low_confidence':
                p = F.softmax(logits.to(torch.float64), dim=-1)
                x0_p = torch.squeeze(torch.gather(p, dim=-1, index=x0.unsqueeze(-1)), -1)
            elif remasking == 'random':
                x0_p = torch.rand_like(x0, device=x0.device)
            else:
                raise NotImplementedError(f"Remasking strategy '{remasking}' not implemented.")

            x0_p[:, prompt.shape[1] + (num_block + 1) * block_length:] = -np.inf
            x0 = torch.where(mask_index, x0, x)
            confidence = torch.where(mask_index, x0_p, -np.inf)

            transfer_index = torch.zeros_like(x0, dtype=torch.bool, device=x0.device)
            for j in range(confidence.shape[0]):
                _, select_index = torch.topk(confidence[j], k=num_transfer_tokens[j, i])
                transfer_index[j, select_index] = True
            x[transfer_index] = x0[transfer_index]

    return x

In [ ]:
# -- Load test data --
df_test = pd.read_csv('/Users/pupipatsingkhorn/Developer/repositories/NanoLLaDA/data/test-100-1024.csv')

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(config.model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(config.model_name, trust_remote_code=True, torch_dtype=torch.bfloat16)
model.to(config.device)
model.eval()

# Function to generate summary for a single input text
@torch.no_grad()
def summarize_text(text):
    try:
        messages = [{"role": "user", "content": f"สรุปข้อความต่อไปนี้\n{text}"}]
        prompt = tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=False
        )
        input_ids = torch.tensor(tokenizer(prompt)["input_ids"]).to(config.device).unsqueeze(0)

        output_ids = generate(
            model,
            input_ids,
            steps=256,
            gen_length=256,
            block_length=32,
            temperature=0.7,
            cfg_scale=0.0,
            remasking="low_confidence",
            mask_id=config.mask_token_id,
        )

        decoded = tokenizer.batch_decode(
            output_ids[:, input_ids.shape[1]:], skip_special_tokens=True
        )[0]
        return decoded.strip()
    except Exception as e:
        return f"Error: {e}"

tqdm.pandas()
df_test['generated'] = df_test['body'].progress_apply(summarize_text)
